TEST ML_composer 


In [1]:
import os
print(os.getcwd())
os.chdir("F:/project/sugarcane_disease/")
#os.chdir("O:/project/sugarcane_disease/")
os.getcwd()
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
try:
    import keras
    from keras.models import Sequential
    from tensorflow.keras import layers
    from keras.layers import MaxPooling1D, Flatten, Dense, Conv1D,MaxPooling2D, Conv2D
    from keras.layers import Dropout
    from tensorflow.keras.utils import to_categorical
    import tensorflow as tf
    import keras.metrics
except:
    try:
        from tensorflow import keras
        from tensorflow.keras import layers
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.utils import to_categorical
        from tensorflow.keras.layers import MaxPooling1D, Flatten, Dense, Conv1D,MaxPooling2D, Conv2D, Dropout
        import tensorflow as tf
        from keras_self_attention import SeqSelfAttention

        print("Use tensorflow backend keras module")
    except:
        print("This is not a GPU env.")
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
import numpy as np
import configparser
from Functions import *
from tensorflow.keras.layers import Layer
from CustomLayers import *
tf.config.experimental_run_functions_eagerly(True)
# Define the residual block as a new layer

E:\learning resource\PhD\PHD_Notebook\Code\ML_composer


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'F:/project/sugarcane_disease/'

In [54]:
import os
from GS_composer import *

In [55]:
from ClassModel import *
from CustomLayers import *

In [56]:
from Functions import *

In [57]:
class p_args:
    
    def __init__(self):
        
        self.ped = "E:/learning resource/OneDrive - The University of Queensland/PhD/data/sugarcane_disease/subset/disease_subset"
        self.pheno = "E:/learning resource/OneDrive - The University of Queensland/PhD/data/sugarcane_disease/subset/disease_subset.phen"
        self.mpheno = 1
        self.index = "E:/learning resource/OneDrive - The University of Queensland/PhD/data/sugarcane_disease/subset/disease_subset.index"
        self.model = "MultiHead Attention LNN"
        self.annotation = None
        self.trait = "smut"
        self.output = "./Attention_Test_win"
        
        self.load = None
        self.save = True
        self.plot = True
        
        self.round = 1
        self.epoch = 10
        self.rank =False
        self.lr = 0.0001
        self.batch = 32
        self.mean = False
        self.width = 64
        self.depth = 2
        self.quiet = 1
        self.num_heads = 1
        self.residual = True
        self.embedding = 12
        
        
args = p_args()

In [58]:
config = None
os.chdir("E:/learning resource/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/")

In [59]:
if args.output[0] == "/":
    locat = '/' + args.output.strip('/') + '/'
else:
    locat = args.output.strip('/') + '/'
if not os.path.exists(locat):
    os.mkdir(locat)

In [60]:
composer = ML_composer()
composer._model
composer.silence_mode = 2

In [61]:
#composer.get_data(config,args)
composer.args = args

In [62]:
composer.get_data(config,args=args)

1    939
3    940
2    940
4    940
5    943
Name: 2, dtype: int64
Get genotype shape: (4702, 100)
   7   8   9   10  11  12  13  14  15
1   0   2   0   1   2   1   0   1   2
2   1   2   1   1   2   1   0   1   1
3   1   2   0   0   1   0   1   1   2
4   1   2   0   0   1   0   1   1   2
5   0   2   1   1   2   0   2   1   1
6   0   2   0   1   2   1   0   1   2
7   1   2   1   1   2   1   2   1   2
8   0   2   0   1   1   0   1   0   2
9   1   2   1   1   2   1   2   1   2
Running data check
GENO
Samples are not aligned with same order
FAM
PHENO
Samples are not aligned with same order
INDEX
Samples are not aligned with same order


In [63]:
composer.prepare_model()

In [64]:
index_ref = composer.prepare_cross_validate()

In [65]:
print("Cross-validate: {}".format(index_ref[0]))

Cross-validate: ([2, 3, 4, 5], [1])


In [66]:
train_idx,valid_idx = index_ref[0]

In [67]:
composer.prepare_training(train_idx,valid_idx)
#composer.batchSize = 64

Overall population: 4702
759 individuals need to be removed due to the miss phenotype
Filtered population: 3943
Mean of train phenotype: 4.715129635707253
Use raw phenotype as the target
16    1.0
39    3.0
44    3.0
47    2.0
92    3.0
Name: 2, dtype: float64


In [68]:
#composer._raw_data["GENO"].iloc[:,6:].describe()

In [69]:
composer.compose(train_idx,valid_idx)

USE Attention CNN MODEL as training method
All the SNPs are already decoded, imputing missing SNPs with 0.01
Convert data to np.array float32
The transformed SNP shape: (3047, 100, 1)
USE Attention CNN MODEL as training method
All the SNPs are already decoded, imputing missing SNPs with 0.01
Convert data to np.array float32
The transformed SNP shape: (896, 100, 1)
Train status:
Epochs:  10
Repeat(Round):  1


TypeError: You are attempting to use Python control flow in a layer that was not declared to be dynamic. Pass `dynamic=True` to the class constructor.
Encountered error:
"""
in user code:

    E:\learning resource\PhD\PHD_Notebook\Code\ML_composer\CustomLayers.py:213 build  *
        assert len(input_shape[0]) >= 2
    C:\Users\pc\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\ops.py:877 __bool__
        self._disallow_bool_casting()
    C:\Users\pc\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\ops.py:490 _disallow_bool_casting
        self._disallow_in_graph_mode("using a `tf.Tensor` as a Python `bool`")
    C:\Users\pc\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\ops.py:479 _disallow_in_graph_mode
        " this function with @tf.function.".format(task))

    OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this f

In [ ]:
#import tensorflow as tf 

if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
1.5//1

In [93]:
os.getcwd()

'E:\\learning resource\\OneDrive - The University of Queensland\\PhD\\HPC_Results\\Sugarcane_disease\\ML'

In [ ]:
index_ref = composer.prepare_cross_validate()
i = 1
for train_idx,valid_idx in index_ref:
    print("Cross-validate: {}".format(i))
    composer.prepare_training(train_idx,valid_idx)
    composer.compose(train_idx,valid_idx)
    i+=1